In [22]:
%load_ext autoreload
%autoreload 2

from src import *
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
batch_size = 1024
epochs = 10
image_feat_path = "./data/test_data/flickr30k/flickr30k_vgg_image_feat_test.mat"
sent_feat_path = "./data/test_data/flickr30k/flickr30k_text_feat_test.mat"
shuffle_step=5

In [3]:
data_loader = DatasetLoader(image_feat_path, sent_feat_path)

Loading image features from ./data/test_data/flickr30k/flickr30k_vgg_image_feat_test.mat
Loaded image feature shape: (1000, 4096)
Loading sentence features from ./data/test_data/flickr30k/flickr30k_text_feat_test.mat
Loaded sentence feature shape: (5000, 6000)


In [7]:
model = embedding_model(data_loader.im_feats.shape, data_loader.sent_feats.shape)
loss = EmbeddingLoss()

In [23]:
img_feats = data_loader.im_feats
sent_feats = data_loader.sent_feats


sent_sample_size = 5 # how many sentences per image

total_loss, batch_loss = [], []
optimizer = tf.keras.optimizers.Adam()

In [25]:
optimizer = tf.keras.optimizers.Adam()

for i in range(1):
    count = i % shuffle_step
    # if count == 0:
    #     data_loader.shuffle_inds()
    im_feats, sent_feats, labels = data_loader.get_batch(count, batch_size, sent_sample_size)
    _batch_loss = tf.constant(0, dtype=tf.float32)
    with tf.GradientTape() as tape:
        [im_emb, sent_emb] = model(im_feats, sent_feats)

        _batch_loss += loss(im_embeds, sent_emb, labels)

    trainable_variables = model.trainable_variables

    gradients = tape.gradient(_batch_loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    # total_loss.append(_batch_loss / int(im_feats.shape[1]))

ValueError: Layer model_2 expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor: shape=(1024, 4096), dtype=float32, numpy=
array([[ -1.6155552 ,  -3.6643739 ,   1.3068848 , ...,  -6.1157217 ,
         -0.67167836,   1.210375  ],
       [ -1.6155552 ,  -3.6643739 ,   1.3068848 , ...,  -6.1157217 ,
         -0.67167836,   1.210375  ],
       [ -1.6155552 ,  -3.6643739 ,   1.3068848 , ...,  -6.1157217 ,
         -0.67167836,   1.210375  ],
       ...,
       [ -0.8937439 ,  -1.1320702 ,  -0.5085614 , ..., -12.828728  ,
         -0.04592862,   2.0970378 ],
       [ -0.8937439 ,  -1.1320702 ,  -0.5085614 , ..., -12.828728  ,
         -0.04592862,   2.0970378 ],
       [ -0.8937439 ,  -1.1320702 ,  -0.5085614 , ..., -12.828728  ,
         -0.04592862,   2.0970378 ]], dtype=float32)>]

In [30]:
# currently the data looks like 

# sample size | img_feats | sent_feats
# ====================================
# 1           | img1      | [s1, s2, s3, s4, s5]
# 2           | img2      | [s1, s2, s3, s4, s5]

# we need it to be 

# sample size | img_feats | sent_feats
# ====================================
# 1           | img1      | s1
# 1           | img1      | s2
# 1           | img1      | s3
# 1           | img1      | s4
# 1           | img1      | s5
# 1           | img2      | s1
# 2           | img2      | s2 and so on

SyntaxError: invalid syntax (<ipython-input-30-aa214f857402>, line 1)

(5, 6000)